# Statistics, Part I

This week, we are going to learn some basic tools for statistical testing that will be useful for making predictions based on your data. In statistics, it is important to determine whether the trends we see in data are statistically significant, or possibly just due to chance. In today's lecture, we will learn about how to fit a regression line to continuous data to help us identify trends.

In [ ]:
# Run this cell first to import libraries and modules
from datascience import *
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error

## Fitting a regression line
Let's say we had the following data. We can plot it using `plt.scatter`:

In [ ]:
x1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y1 = [2, 5, 8, 7, 7, 9, 12, 13, 15, 14]

plt.scatter(x1, y1, color='blue')
plt.xlabel('x values')
_ = plt.ylabel('y values') # the _ = code removes the output text

Now let's try to add a line of best fit. By "best" here we mean the line that best accounts for the relationship between x and y. I randomly came up with the values in `guess`:

In [ ]:
guess = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
plt.scatter(x1, y1, color='blue')
plt.plot(x1, guess, color='black')
plt.xlabel('x values')
_ = plt.ylabel('y values') # the _ = code removes the output text

Not a very good guess. This line doesn't "fit" very well. What if we could automatically calculate the formula for the line of best fit? The function `np.polyfit` can help us do just that.

`np.polyfit` has three required arguments:

- an array of x-values
- an array of y-values
- the degree of the fitting polynomial (`deg`)

It returns a `numpy` array with:

- the `deg` polynomial coefficients
- the intercept

In this example, we will use `deg=1`, because we simply want a straight line. That is, we want a one-degree polynomial:

$$\hat{y} = slope * x + intercept$$

`numpy` returns the polynomial coefficient(s), the last of which is the intercept: 

In [ ]:
output = np.polyfit(x1, y1, deg=1)
print(output)

More transparently, we can write the following:

In [ ]:
slope, intercept = np.polyfit(x1, y1, deg=1)
print(slope, intercept)

We can compute the predicted values for the fitted line with the following command: `np.multiply(slope, x_values) + intercept`

In [ ]:
np.multiply(slope, x1)

In [ ]:
np.multiply(slope, x1) + intercept

This makes it easy to plot the line of best fit:

In [ ]:
# plot the points
plt.scatter(x1, y1, color='blue')
# compute the line of best fit
fit1 = intercept + np.multiply(slope, x1)
# plot the fitted line
plt.plot(x1, fit1, color='black')
# plot the old guess line
plt.plot(x1, guess, color='red')
# add the axis labels
plt.xlabel('x values')
_ = plt.ylabel('y values') # the _ = code removes the output text

What if we had data like the following?

In [ ]:
x2 = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
y2 = [-125, -64, -10, -8, -1, 0, 1, 8, 10, 64, 125]
plt.scatter(x2, y2, color='green')
plt.xlabel('x values')
_ = plt.ylabel('y values') # the _ = code removes the output text

A straight line doesn't appear to be a good fit:

In [ ]:
# plot the points
plt.scatter(x2, y2, color='green')
# compute the line of best fit
slope, intercept = np.polyfit(x2, y2, deg=1)
fit2 = intercept + np.multiply(slope, x2)
# plot the fitted line
plt.plot(x2, fit2, '-', color='black')
plt.xlabel('x values')
_ = plt.ylabel('y values')

We can get a better fit by increasing `deg`. If we use `deg=3` then we get an array with four values:

In [ ]:
out = np.polyfit(x2, y2, deg=3)
print(out)

More formally, an array `z` is returned with the values `[z3, z2, z1, z0]`, as described below:

$$\hat{y} = z_0x^0 + z_1x^1 + z_2x^2 + z_3x^3$$

We can then compute the new fitted line and add it to our plot. Note that `np.polyfit` returns the coefficients "in reverse":

In [ ]:
# get a new fitted line
z3, z2, z1, z0 = np.polyfit(x2, y2, deg=3)
# store the result
fit3 = z0 + np.multiply(z1, x2) + np.multiply(z2, np.power(x2, 2)) + np.multiply(z3, np.power(x2, 3))

The new fitted line is clearly much better than the straight line fit:

In [ ]:
# plot the points
plt.scatter(x2, y2, color='green')
# plot the old fitted line
plt.plot(x2, fit2, '-', color='red')
# plot the new fitted line
plt.plot(x2, fit3, '-', color='black')
plt.xlabel('x values')
_ = plt.ylabel('y values')

In data science, we use best fit lines like these to make accurate predictions about real-world phenomena based on the data we have. Let's try doing this with real data.

## Psycholinguistics and lexical decision

The data for this demo is a lexical decision dataset. It was elicited from 21 subjects for 79 English concrete nouns. RT stands for "response time" (in what I assume is a normalized scale, rather than seconds). Frequency indicates a calculated frequency for the noun within a given corpus. In this lexical decision task, subjects were shown a series of words (e.g., CAT, BAT, CART) and the occasional nonword (e.g., CAZ, BRIT, CHOG) and asked to simply identify whether the word was real or not. In addition to measuring accuracy, recording the response time can tell us a bit about how our brains process words, as faster (lower) response times result from faster, easier mental processing.

In [ ]:
lex = Table.read_table('wk4-lexicaldecision.csv')
lex.show(5)

In [ ]:
set(lex.column('Word'))

In [ ]:
lex.where('Word','owl')

### Tangents: random useful functions and methods

A random useful `numpy` function for numeric arrays is `np.count_nonzero(x==y)`.

In [ ]:
wordlength = lex.column('Length')
np.count_nonzero(wordlength==3)

Don't forget the built-in `numpy` functions that give us basic statistical summaries.

In [ ]:
r = np.mean(lex.column('RT'))
print("The average response time for respondents was",r.round(2))

Finally, I'll demonstrate how to take a sample of a Table, which might be necessary in the event that you have a Table that is many thousands of rows long. Not so necessary for this dataset, but maybe in your homework...

In [ ]:
lex_s = lex.sample(100,with_replacement=False)
lex_s

### Back to the Lines of Best Fit...

Let's analyze whether response time (RT) is related in some way to word frequency. We can visualize each of these columns separately first and look at the distribution of values in a histogram.

In [ ]:
lex.hist('RT')

In [ ]:
lex.hist('Frequency')

Then, we plot our scatter plot.

In [ ]:
plt.scatter(data=lex,x="Frequency",y="RT")
plt.xlabel('Frequency of word')
_ = plt.ylabel('Response Time (standardized-sec)')

Let's plot a line of best fit over this data. Do you think the slope of the line will be positive or negative?

In [ ]:
xvals = lex.column('Frequency')
yvals = lex.column('RT')
# equivalent to getting our regression line, which involves a slope and an intercept
slope, intercept = np.polyfit(xvals, yvals, 1)
print(slope, intercept)

In [ ]:
# y_predicted = slope * x + intercept
lbf = np.multiply(slope, xvals) + intercept
lbf

In [ ]:
plt.scatter(data=lex,x="Frequency",y="RT")
plt.plot(xvals,lbf, color='red')
plt.title('Correlation between word frequency and response time')
plt.xlabel('Frequency of word')
_ = plt.ylabel('Response Time (standardized-sec)')

What we are identifying here is called a *correlation*: a measure of the strength of the relationship between two continuous variables: e.g., as *x* increases, *y* decreases. Correlation is a common measure of association. Other measures like this include regression, odds ratio, and chi-square tests. In Wednesday's lecture, we'll talk about testing correlation for statistical significance.